In [1]:
import geopandas as gpd

wards = gpd.read_file("../geo/delhi_wards.geojson")

wards = wards.to_crs(epsg=32643)

wards["area_km2"] = wards.geometry.area / 1e6
print("Wards area calculated :)")

Wards area calculated :)


In [ ]:
import pandas as pd
import geopandas as gpd
import re

pop_df = pd.read_csv("../data/external/delhi-ward-wise-population.csv")

pop_df["Ward_No_Clean"] = pop_df["Ward"].str.extract(r'(\d+)$')
pop_df["Ward_No_Clean"] = pop_df["Ward_No_Clean"].astype(float)  

wards = gpd.read_file("../geo/delhi_wards.geojson")

wards["Ward_No_Clean"] = wards["Ward_No"].apply(
    lambda x: float(re.search(r"\d+", str(x)).group()) if pd.notnull(x) and re.search(r"\d+", str(x)) else None
)


wards = wards.to_crs(epsg=32643)

wards["area_km2"] = wards.geometry.area / 1e6

merged = wards.merge(pop_df[["Ward_No_Clean", "Population"]], on="Ward_No_Clean", how="left")

merged["population_density"] = merged["Population"] / merged["area_km2"]

merged.to_file("../data/processed/wards_with_population.geojson", driver="GeoJSON")
merged[["Ward_Name", "Population", "area_km2", "population_density"]].to_csv(
    "../data/processed/wards_with_population.csv", index=False
)
